## <u>2. Datenaufbereitung Previous Application


*Vorgehensweise kategorische Variablen:*
- Gruppierung der Variablen
- Löschung von kategorischen Variablen mit weniger als 5pP relativer Anteilsdifferenz zwischen Paybacks und Defaults
- Löschung von Variablen mit mehr als 60% fehlenden Daten

## Initialisierung

In [1]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from sklearn.linear_model import LogisticRegression

from IPython.display import display, Markdown

In [2]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [3]:
pa = pd.read_csv(DATASET_DIR / "previous_application.csv")
pos = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "pos.csv")
ip = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "ip.csv")
cc = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "cc.csv")
description = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [4]:
pa = pa.set_index("SK_ID_PREV")
pos = pos.set_index("SK_ID_PREV")
ip = ip.set_index("SK_ID_PREV")
cc = cc.set_index("SK_ID_PREV")

In [5]:
pa.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# Spalten die innerhalb der Aufbereitung nicht verändert werden können
skip = ["SK_ID_CURR", "SK_ID_PREV"]

In [7]:
# nominale und metrische Spalten
n_heads = [element for element in pa.columns if pa[element].dtype.name == "object"]
n_heads += ["NFLAG_LAST_APPL_IN_DAY", "NFLAG_INSURED_ON_APPROVAL"]
o_heads = ["HOUR_APPR_PROCESS_START", "SELLERPLACE_AREA"]
m_heads = [element for element in pa.columns if element not in n_heads and element not in o_heads]

## <u> kategorische Variablen

In [8]:
df = pa[["SK_ID_CURR"] + n_heads].copy()

In [9]:
result = pd.DataFrame(index=pa.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

In [10]:
df.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,WEEKDAY_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,NAME_SELLER_INDUSTRY,NAME_YIELD_GROUP,PRODUCT_COMBINATION,NFLAG_LAST_APPL_IN_DAY,NFLAG_INSURED_ON_APPROVAL
SK_ID_PREV,,,,,,,,,,,,,,,,,,,
2030495,271877,Consumer loans,SATURDAY,Y,XAP,Approved,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,Connectivity,middle,POS mobile with interest,1,0.0
2802425,108129,Cash loans,THURSDAY,Y,XNA,Approved,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,XNA,low_action,Cash X-Sell: low,1,1.0
2523466,122040,Cash loans,TUESDAY,Y,XNA,Approved,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,high,Cash X-Sell: high,1,1.0
2819243,176158,Cash loans,MONDAY,Y,XNA,Approved,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,XNA,middle,Cash X-Sell: middle,1,1.0
1784265,202054,Cash loans,THURSDAY,Y,Repairs,Refused,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,XNA,high,Cash Street: high,1,NaN


### Aufsummierung aller historischen Kredite nach Kategorie

In [11]:
for cat in df.columns.values:
    if cat in skip:
        continue
    
    A = df[["SK_ID_CURR", cat]].groupby(by=["SK_ID_CURR", cat]).size().unstack(fill_value=0)
    A.columns = ["CNT_{cat}_{head}".format(cat=cat, head=head) for head in A.keys().values]
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

In [12]:
df = result
df.head()

,CNT_NAME_CONTRACT_TYPE_Cash loans,CNT_NAME_CONTRACT_TYPE_Consumer loans,CNT_NAME_CONTRACT_TYPE_Revolving loans,CNT_NAME_CONTRACT_TYPE_XNA,CNT_WEEKDAY_APPR_PROCESS_START_FRIDAY,CNT_WEEKDAY_APPR_PROCESS_START_MONDAY,CNT_WEEKDAY_APPR_PROCESS_START_SATURDAY,CNT_WEEKDAY_APPR_PROCESS_START_SUNDAY,CNT_WEEKDAY_APPR_PROCESS_START_THURSDAY,CNT_WEEKDAY_APPR_PROCESS_START_TUESDAY,CNT_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,CNT_FLAG_LAST_APPL_PER_CONTRACT_N,CNT_FLAG_LAST_APPL_PER_CONTRACT_Y,CNT_NAME_CASH_LOAN_PURPOSE_Building a house or an annex,CNT_NAME_CASH_LOAN_PURPOSE_Business development,CNT_NAME_CASH_LOAN_PURPOSE_Buying a garage,CNT_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,CNT_NAME_CASH_LOAN_PURPOSE_Buying a home,CNT_NAME_CASH_LOAN_PURPOSE_Buying a new car,CNT_NAME_CASH_LOAN_PURPOSE_Buying a used car,CNT_NAME_CASH_LOAN_PURPOSE_Car repairs,CNT_NAME_CASH_LOAN_PURPOSE_Education,CNT_NAME_CASH_LOAN_PURPOSE_Everyday expenses,CNT_NAME_CASH_LOAN_PURPOSE_Furniture,CNT_NAME_CASH_LOAN_PURPOSE_Gasification / water supply,CNT_NAME_CASH_LOAN_PURPOSE_Hobby,CNT_NAME_CASH_LOAN_PURPOSE_Journey,CNT_NAME_CASH_LOAN_PURPOSE_Medicine,CNT_NAME_CASH_LOAN_PURPOSE_Money for a third person,CNT_NAME_CASH_LOAN_PURPOSE_Other,CNT_NAME_CASH_LOAN_PURPOSE_Payments on other loans,CNT_NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment,CNT_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal,CNT_NAME_CASH_LOAN_PURPOSE_Repairs,CNT_NAME_CASH_LOAN_PURPOSE_Urgent needs,CNT_NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday,CNT_NAME_CASH_LOAN_PURPOSE_XAP,CNT_NAME_CASH_LOAN_PURPOSE_XNA,CNT_NAME_CONTRACT_STATUS_Approved,CNT_NAME_CONTRACT_STATUS_Canceled,CNT_NAME_CONTRACT_STATUS_Refused,CNT_NAME_CONTRACT_STATUS_Unused offer,CNT_NAME_PAYMENT_TYPE_Cash through the bank,CNT_NAME_PAYMENT_TYPE_Cashless from the account of the employer,CNT_NAME_PAYMENT_TYPE_Non-cash from your account,CNT_NAME_PAYMENT_TYPE_XNA,CNT_CODE_REJECT_REASON_CLIENT,CNT_CODE_REJECT_REASON_HC,CNT_CODE_REJECT_REASON_LIMIT,CNT_CODE_REJECT_REASON_SCO,CNT_CODE_REJECT_REASON_SCOFR,CNT_CODE_REJECT_REASON_SYSTEM,CNT_CODE_REJECT_REASON_VERIF,CNT_CODE_REJECT_REASON_XAP,CNT_CODE_REJECT_REASON_XNA,CNT_NAME_TYPE_SUITE_Children,CNT_NAME_TYPE_SUITE_Family,CNT_NAME_TYPE_SUITE_Group of people,CNT_NAME_TYPE_SUITE_Other_A,CNT_NAME_TYPE_SUITE_Other_B,"CNT_NAME_TYPE_SUITE_Spouse, partner",CNT_NAME_TYPE_SUITE_Unaccompanied,CNT_NAME_CLIENT_TYPE_New,CNT_NAME_CLIENT_TYPE_Refreshed,CNT_NAME_CLIENT_TYPE_Repeater,CNT_NAME_CLIENT_TYPE_XNA,CNT_NAME_GOODS_CATEGORY_Additional Service,CNT_NAME_GOODS_CATEGORY_Animals,CNT_NAME_GOODS_CATEGORY_Audio/Video,CNT_NAME_GOODS_CATEGORY_Auto Accessories,CNT_NAME_GOODS_CATEGORY_Clothing and Accessories,CNT_NAME_GOODS_CATEGORY_Computers,CNT_NAME_GOODS_CATEGORY_Construction Materials,CNT_NAME_GOODS_CATEGORY_Consumer Electronics,CNT_NAME_GOODS_CATEGORY_Direct Sales,CNT_NAME_GOODS_CATEGORY_Education,CNT_NAME_GOODS_CATEGORY_Fitness,CNT_NAME_GOODS_CATEGORY_Furniture,CNT_NAME_GOODS_CATEGORY_Gardening,CNT_NAME_GOODS_CATEGORY_Homewares,CNT_NAME_GOODS_CATEGORY_House Construction,CNT_NAME_GOODS_CATEGORY_Insurance,CNT_NAME_GOODS_CATEGORY_Jewelry,CNT_NAME_GOODS_CATEGORY_Medical Supplies,CNT_NAME_GOODS_CATEGORY_Medicine,CNT_NAME_GOODS_CATEGORY_Mobile,CNT_NAME_GOODS_CATEGORY_Office Appliances,CNT_NAME_GOODS_CATEGORY_Other,CNT_NAME_GOODS_CATEGORY_Photo / Cinema Equipment,CNT_NAME_GOODS_CATEGORY_Sport and Leisure,CNT_NAME_GOODS_CATEGORY_Tourism,CNT_NAME_GOODS_CATEGORY_Vehicles,CNT_NAME_GOODS_CATEGORY_Weapon,CNT_NAME_GOODS_CATEGORY_XNA,CNT_NAME_PORTFOLIO_Cards,CNT_NAME_PORTFOLIO_Cars,CNT_NAME_PORTFOLIO_Cash,CNT_NAME_PORTFOLIO_POS,CNT_NAME_PORTFOLIO_XNA,CNT_NAME_PRODUCT_TYPE_XNA,CNT_NAME_PRODUCT_TYPE_walk-in,CNT_NAME_PRODUCT_TYPE_x-sell,CNT_CHANNEL_TYPE_AP+ (Cash loan),CNT_CHANNEL_TYPE_Car dealer,CNT_CHANNEL_TYPE_Channel of corporate sales,CNT_CHANNEL_TYPE_Contact center,CNT_CHANNEL_TYPE_Country-wide,CNT_CHANNEL_TYPE_Credit and cash offices,CNT_CHANNEL_TYPE_Regional / Local,CNT_CHANNEL_TYPE_Stone,CNT_NAME_SELLER_INDUSTR

Kreditnehmer 271877 hat in seiner Kredithistorie von Home Credit 3 Konsumentenkredite beantragt, wovon 2 genehmigt und 1 abgelehnt wurde, etc. 

### Speichern der kategorischen Werte

In [13]:
cats = df

## <u> metrische Variablen

In [14]:
ID_PA_POS = np.intersect1d(pa.index.values, pos.index.values)
ID_PA_IP = np.intersect1d(pa.index.values, ip.index.values)
ID_PA_CC = np.intersect1d(pa.index.values, cc.index.values)

# Ermittlung der Kredite ohne Kredithistorie
# -> PA's, die nicht in POS, IP oder CC sind

A = np.concatenate((ID_PA_POS,ID_PA_IP,ID_PA_CC)) 
B = pa.drop(A)

ID_PA_ONLY = B.index.values

In [15]:
# Erstellung der Unterdatensätze je nach Kredittyp

In [16]:
pa_pos = pa.loc[ID_PA_POS]
pa_ip = pa.loc[ID_PA_IP]
pa_cc = pa.loc[ID_PA_CC]
pa_only = pa.loc[ID_PA_ONLY]

In [17]:
pa_pos = pd.merge(pa_pos, pos, how="left", left_index=True, right_index=True)
pa_ip = pd.merge(pa_ip, ip, how="left", left_index=True, right_index=True)
pa_cc = pd.merge(pa_cc, cc, how="left", left_index=True, right_index=True)

In [18]:
df.head()

,CNT_NAME_CONTRACT_TYPE_Cash loans,CNT_NAME_CONTRACT_TYPE_Consumer loans,CNT_NAME_CONTRACT_TYPE_Revolving loans,CNT_NAME_CONTRACT_TYPE_XNA,CNT_WEEKDAY_APPR_PROCESS_START_FRIDAY,CNT_WEEKDAY_APPR_PROCESS_START_MONDAY,CNT_WEEKDAY_APPR_PROCESS_START_SATURDAY,CNT_WEEKDAY_APPR_PROCESS_START_SUNDAY,CNT_WEEKDAY_APPR_PROCESS_START_THURSDAY,CNT_WEEKDAY_APPR_PROCESS_START_TUESDAY,CNT_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,CNT_FLAG_LAST_APPL_PER_CONTRACT_N,CNT_FLAG_LAST_APPL_PER_CONTRACT_Y,CNT_NAME_CASH_LOAN_PURPOSE_Building a house or an annex,CNT_NAME_CASH_LOAN_PURPOSE_Business development,CNT_NAME_CASH_LOAN_PURPOSE_Buying a garage,CNT_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,CNT_NAME_CASH_LOAN_PURPOSE_Buying a home,CNT_NAME_CASH_LOAN_PURPOSE_Buying a new car,CNT_NAME_CASH_LOAN_PURPOSE_Buying a used car,CNT_NAME_CASH_LOAN_PURPOSE_Car repairs,CNT_NAME_CASH_LOAN_PURPOSE_Education,CNT_NAME_CASH_LOAN_PURPOSE_Everyday expenses,CNT_NAME_CASH_LOAN_PURPOSE_Furniture,CNT_NAME_CASH_LOAN_PURPOSE_Gasification / water supply,CNT_NAME_CASH_LOAN_PURPOSE_Hobby,CNT_NAME_CASH_LOAN_PURPOSE_Journey,CNT_NAME_CASH_LOAN_PURPOSE_Medicine,CNT_NAME_CASH_LOAN_PURPOSE_Money for a third person,CNT_NAME_CASH_LOAN_PURPOSE_Other,CNT_NAME_CASH_LOAN_PURPOSE_Payments on other loans,CNT_NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment,CNT_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal,CNT_NAME_CASH_LOAN_PURPOSE_Repairs,CNT_NAME_CASH_LOAN_PURPOSE_Urgent needs,CNT_NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday,CNT_NAME_CASH_LOAN_PURPOSE_XAP,CNT_NAME_CASH_LOAN_PURPOSE_XNA,CNT_NAME_CONTRACT_STATUS_Approved,CNT_NAME_CONTRACT_STATUS_Canceled,CNT_NAME_CONTRACT_STATUS_Refused,CNT_NAME_CONTRACT_STATUS_Unused offer,CNT_NAME_PAYMENT_TYPE_Cash through the bank,CNT_NAME_PAYMENT_TYPE_Cashless from the account of the employer,CNT_NAME_PAYMENT_TYPE_Non-cash from your account,CNT_NAME_PAYMENT_TYPE_XNA,CNT_CODE_REJECT_REASON_CLIENT,CNT_CODE_REJECT_REASON_HC,CNT_CODE_REJECT_REASON_LIMIT,CNT_CODE_REJECT_REASON_SCO,CNT_CODE_REJECT_REASON_SCOFR,CNT_CODE_REJECT_REASON_SYSTEM,CNT_CODE_REJECT_REASON_VERIF,CNT_CODE_REJECT_REASON_XAP,CNT_CODE_REJECT_REASON_XNA,CNT_NAME_TYPE_SUITE_Children,CNT_NAME_TYPE_SUITE_Family,CNT_NAME_TYPE_SUITE_Group of people,CNT_NAME_TYPE_SUITE_Other_A,CNT_NAME_TYPE_SUITE_Other_B,"CNT_NAME_TYPE_SUITE_Spouse, partner",CNT_NAME_TYPE_SUITE_Unaccompanied,CNT_NAME_CLIENT_TYPE_New,CNT_NAME_CLIENT_TYPE_Refreshed,CNT_NAME_CLIENT_TYPE_Repeater,CNT_NAME_CLIENT_TYPE_XNA,CNT_NAME_GOODS_CATEGORY_Additional Service,CNT_NAME_GOODS_CATEGORY_Animals,CNT_NAME_GOODS_CATEGORY_Audio/Video,CNT_NAME_GOODS_CATEGORY_Auto Accessories,CNT_NAME_GOODS_CATEGORY_Clothing and Accessories,CNT_NAME_GOODS_CATEGORY_Computers,CNT_NAME_GOODS_CATEGORY_Construction Materials,CNT_NAME_GOODS_CATEGORY_Consumer Electronics,CNT_NAME_GOODS_CATEGORY_Direct Sales,CNT_NAME_GOODS_CATEGORY_Education,CNT_NAME_GOODS_CATEGORY_Fitness,CNT_NAME_GOODS_CATEGORY_Furniture,CNT_NAME_GOODS_CATEGORY_Gardening,CNT_NAME_GOODS_CATEGORY_Homewares,CNT_NAME_GOODS_CATEGORY_House Construction,CNT_NAME_GOODS_CATEGORY_Insurance,CNT_NAME_GOODS_CATEGORY_Jewelry,CNT_NAME_GOODS_CATEGORY_Medical Supplies,CNT_NAME_GOODS_CATEGORY_Medicine,CNT_NAME_GOODS_CATEGORY_Mobile,CNT_NAME_GOODS_CATEGORY_Office Appliances,CNT_NAME_GOODS_CATEGORY_Other,CNT_NAME_GOODS_CATEGORY_Photo / Cinema Equipment,CNT_NAME_GOODS_CATEGORY_Sport and Leisure,CNT_NAME_GOODS_CATEGORY_Tourism,CNT_NAME_GOODS_CATEGORY_Vehicles,CNT_NAME_GOODS_CATEGORY_Weapon,CNT_NAME_GOODS_CATEGORY_XNA,CNT_NAME_PORTFOLIO_Cards,CNT_NAME_PORTFOLIO_Cars,CNT_NAME_PORTFOLIO_Cash,CNT_NAME_PORTFOLIO_POS,CNT_NAME_PORTFOLIO_XNA,CNT_NAME_PRODUCT_TYPE_XNA,CNT_NAME_PRODUCT_TYPE_walk-in,CNT_NAME_PRODUCT_TYPE_x-sell,CNT_CHANNEL_TYPE_AP+ (Cash loan),CNT_CHANNEL_TYPE_Car dealer,CNT_CHANNEL_TYPE_Channel of corporate sales,CNT_CHANNEL_TYPE_Contact center,CNT_CHANNEL_TYPE_Country-wide,CNT_CHANNEL_TYPE_Credit and cash offices,CNT_CHANNEL_TYPE_Regional / Local,CNT_CHANNEL_TYPE_Stone,CNT_NAME_SELLER_INDUSTR

## PA_POS

In [19]:
# pa_pos

In [20]:
df = pa_pos
df = df.drop(n_heads + o_heads, axis=1)
df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,POS_CNT_PAYMENTS_LEFT,POS_CNT_DPD
SK_ID_PREV,,,,,,,,,,,,,,,,,,
1000001,158271,6404.310,58905.000,65124.0,0.000,58905.000,0.000000,NaN,NaN,-299,12.0,365243.0,-268.0,62.0,-238.0,-233.0,0.0,0.0
1000002,101962,6264.000,39145.500,35230.5,3915.000,39145.500,0.108922,NaN,NaN,-1634,6.0,365243.0,-1600.0,-1450.0,-1510.0,-1501.0,0.0,0.0
1000003,252457,4951.350,47056.275,52641.0,4.275,47056.275,0.000088,NaN,NaN,-124,12.0,365243.0,-94.0,236.0,365243.0,365243.0,9.0,0.0
1000004,260094,3391.110,35144.370,30586.5,7032.870,35144.370,0.203603,NaN,NaN,-893,10.0,365243.0,-862.0,-592.0,-682.0,-672.0,0.0,0.0
1000005,176456,14713.605,123486.075,120307.5,12349.575,123486.075,0.101388,NaN,NaN,-1719,10.0,365243.0,-1688.0,-1418.0,-1418.0,-1415.0,0.0,0.0


In [21]:
result = pd.DataFrame(index=pa_pos.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

### Gruppierung

In [22]:
# count
A = df[["SK_ID_CURR"]]
A.reset_index(level=0, inplace=True)
A = A.groupby(by=["SK_ID_CURR"]).count()
A.columns = ["CNT_CREDITS"]
result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Anzahl der vergangenen POS Kredite

In [23]:
# sums
sum_heads = ["POS_CNT_PAYMENTS_LEFT", "POS_CNT_DPD"] + ["AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE", "RATE_DOWN_PAYMENT"]

for head in sum_heads:
    A = df[["SK_ID_CURR", head]].groupby(by=["SK_ID_CURR"]).sum()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Hat ein Kreditnehmer mehrere Kredite paralell werden deren Daten aufsummiert um die gesamte jährliche Zahlungshöhe, Kredithöhe etc zu ermitteln.

### Zusammenfügen und Speichern der PA_POS Daten

In [24]:
pa_pos = result

In [25]:
pa_pos = pd.merge(pa_pos, cats, how="left", left_index=True, right_index=True)

In [26]:
pa_pos = pa_pos.add_prefix("PA_")

In [27]:
pa_pos = pa_pos.sort_index()

In [28]:
pa_pos2 = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_pos.csv",index_col=0)

In [29]:
pa_pos = pa_pos[pa_pos2.columns.values]

In [30]:
pa_pos.to_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_pos_all.csv")

## PA_IP

In [31]:
# pa_ip

In [32]:
df = pa_ip
df = df.drop(n_heads + o_heads, axis=1)
df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,IP_PAYMENT_30,IP_DAYS_DIFF_30,IP_AMT_DIFF_30,IP_PAYMENT_60,IP_DAYS_DIFF_60,IP_AMT_DIFF_60,IP_PAYMENT_90,IP_DAYS_DIFF_90,IP_AMT_DIFF_90,IP_PAYMENT_120,IP_DAYS_DIFF_120,IP_AMT_DIFF_120,IP_PAYMENT_150,IP_DAYS_DIFF_150,IP_AMT_DIFF_150,IP_PAYMENT_180,IP_DAYS_DIFF_180,IP_AMT_DIFF_180
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000003,252457,4951.350,47056.275,52641.0,4.275,47056.275,0.000088,NaN,NaN,-124,12.0,365243.0,-94.0,236.0,365243.0,365243.0,-0.912871,0.0,0.0,0.912871,-15.0,0.0,0.912871,-17.0,0.0,0.912871,-14.0,0.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0
1000007,256657,11246.805,78570.000,62856.0,15714.000,78570.000,0.217818,NaN,NaN,-158,6.0,365243.0,-123.0,27.0,365243.0,365243.0,0.408248,-7.0,0.0,0.408248,-21.0,0.0,0.408248,-17.0,0.0,0.408248,-19.0,0.0,0.408248,-20.0,0.0,-2.041241,0.0,0.0
1000011,198678,92435.040,855000.000,879831.0,NaN,855000.000,NaN,NaN,NaN,-465,12.0,365243.0,-435.0,-105.0,-105.0,-101.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0,0.912056,-2.0,0.0,0.913278,1.0,0.0,0.913278,-1.0,0.0
1000017,310743,7002.720,67864.230,75024.0,4.230,67864.230,0.000061,NaN,NaN,-516,12.0,365243.0,-486.0,-156.0,-156.0,-151.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,2.041241,-5.0,0.0
1000018,394447,2250.000,45000.000,45000.0,NaN,45000.000,NaN,NaN,NaN,-176,0.0,-175.0,-130.0,365243.0,365243.0,365243.0,0.650875,0.0,0.0,0.814828,-1.0,0.0,-1.044958,-1.0,0.0,-0.867506,-2.0,0.0,-0.768616,-2.0,0.0,1.215377,0.0,0.0


In [33]:
result = pd.DataFrame(index=pa_ip.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

In [34]:
# count
A = df[["SK_ID_CURR"]]
A.reset_index(level=0, inplace=True)
A = A.groupby(by=["SK_ID_CURR"]).count()
A.columns = ["CNT_CREDITS"]
result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Anzahl der vergangenen Install Payments Kredite

In [35]:
ip.columns.values

array(['IP_PAYMENT_30', 'IP_DAYS_DIFF_30', 'IP_AMT_DIFF_30',
       'IP_PAYMENT_60', 'IP_DAYS_DIFF_60', 'IP_AMT_DIFF_60',
       'IP_PAYMENT_90', 'IP_DAYS_DIFF_90', 'IP_AMT_DIFF_90',
       'IP_PAYMENT_120', 'IP_DAYS_DIFF_120', 'IP_AMT_DIFF_120',
       'IP_PAYMENT_150', 'IP_DAYS_DIFF_150', 'IP_AMT_DIFF_150',
       'IP_PAYMENT_180', 'IP_DAYS_DIFF_180', 'IP_AMT_DIFF_180'],
      dtype=object)

In [36]:
# means
mean_heads = list(ip.columns.values)

for head in mean_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).mean()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Aus mehereren Zeitreihen wird der Durchschnitt gebildet, da die Daten normalisiert sind

In [37]:
# sums
sums_heads = ["AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_DOWN_PAYMENT", "AMT_GOODS_PRICE", "RATE_DOWN_PAYMENT"]

for head in sums_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).sum()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Hat ein Kreditnehmer mehrere Kredite paralell werden deren Daten aufsummiert um die gesamte jährliche Zahlungshöhe, Kredithöhe etc zu ermitteln.

In [38]:
result.head()

,CNT_CREDITS,IP_PAYMENT_30,IP_DAYS_DIFF_30,IP_AMT_DIFF_30,IP_PAYMENT_60,IP_DAYS_DIFF_60,IP_AMT_DIFF_60,IP_PAYMENT_90,IP_DAYS_DIFF_90,IP_AMT_DIFF_90,IP_PAYMENT_120,IP_DAYS_DIFF_120,IP_AMT_DIFF_120,IP_PAYMENT_150,IP_DAYS_DIFF_150,IP_AMT_DIFF_150,IP_PAYMENT_180,IP_DAYS_DIFF_180,IP_AMT_DIFF_180,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,
252457,1,-9.128709e-01,0.0,0.0,9.128709e-01,-15.0,0.0,9.128709e-01,-17.0,0.0,0.912871,-14.0,0.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0,4951.350,47056.275,52641.0,4.275,47056.275,0.000088
256657,2,2.041241e-01,-6.0,0.0,2.041241e-01,-13.0,0.0,2.041241e-01,-12.0,0.0,0.204124,-12.5,0.0,0.204124,-13.0,0.0,-1.020621,-1.5,0.0,49054.770,978570.000,1198062.0,15714.000,978570.000,0.217818
198678,2,4.540312e-08,-20.0,0.0,4.540312e-08,-21.5,0.0,4.540312e-08,-6.5,0.0,-0.000407,-1.0,0.0,0.000204,0.5,0.0,0.000204,-0.5,0.0,98806.050,921721.500,941661.0,13347.000,921721.500,0.193358
310743,1,-4.082483e-01,0.0,0.0,-4.082483e-01,0.0,0.0,-4.082483e-01,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,2.041241,-5.0,0.0,7002.720,67864.230,75024.0,4.230,67864.230,0.000061
394447,2,5.295619e-01,-0.5,0.0,6.115381e-01,-7.0,0.0,-3.183548e-01,-7.0,0.0,-0.229629,-6.5,0.0,-0.180184,-8.5,0.0,-0.412932,0.0,0.0,17141.445,129132.000,129132.0,0.000,129132.000,0.000000


### Zusammenführen und Speichern der Daten

In [39]:
pa_ip = result

In [40]:
pa_ip = pd.merge(pa_ip, cats, how="left", left_index=True, right_index=True)

In [41]:
pa_ip = pa_ip.add_prefix("PA_")

In [42]:
pa_ip2 = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_ip.csv",index_col=0)

In [43]:
pa_ip = pa_ip[pa_ip2.columns.values]

In [44]:
pa_ip.head()

,PA_CNT_CREDITS,PA_IP_PAYMENT_30,PA_IP_DAYS_DIFF_30,PA_IP_AMT_DIFF_30,PA_IP_PAYMENT_60,PA_IP_DAYS_DIFF_60,PA_IP_AMT_DIFF_60,PA_IP_PAYMENT_90,PA_IP_DAYS_DIFF_90,PA_IP_AMT_DIFF_90,PA_IP_PAYMENT_120,PA_IP_DAYS_DIFF_120,PA_IP_AMT_DIFF_120,PA_IP_PAYMENT_150,PA_IP_DAYS_DIFF_150,PA_IP_AMT_DIFF_150,PA_IP_PAYMENT_180,PA_IP_DAYS_DIFF_180,PA_IP_AMT_DIFF_180,PA_AMT_ANNUITY,PA_AMT_APPLICATION,PA_AMT_CREDIT,PA_AMT_DOWN_PAYMENT,PA_AMT_GOODS_PRICE,PA_RATE_DOWN_PAYMENT,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_CODE_REJECT_REASON_HC,PA_CNT_NAME_CLIENT_TYPE_New,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_CNT_NAME_PORTFOLIO_Cards,PA_CNT_NAME_PRODUCT_TYPE_walk-in,PA_CNT_NAME_YIELD_GROUP_high,PA_CNT_NAME_YIELD_GROUP_low_normal,PA_CNT_PRODUCT_COMBINATION_Card Street,PA_CNT_PRODUCT_COMBINATION_Cash X-Sell: low,PA_CNT_PRODUCT_COMBINATION_POS industry with interest
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
252457,1,-9.128709e-01,0.0,0.0,9.128709e-01,-15.0,0.0,9.128709e-01,-17.0,0.0,0.912871,-14.0,0.0,-0.912871,0.0,0.0,-0.912871,0.0,0.0,4951.350,47056.275,52641.0,4.275,47056.275,0.000088,1,0,0,0,2,1,1,0,0,1,0,0
256657,2,2.041241e-01,-6.0,0.0,2.041241e-01,-13.0,0.0,2.041241e-01,-12.0,0.0,0.204124,-12.5,0.0,0.204124,-13.0,0.0,-1.020621,-1.5,0.0,49054.770,978570.000,1198062.0,15714.000,978570.000,0.217818,2,7,5,1,0,2,5,0,2,1,0,0
198678,2,4.540312e-08,-20.0,0.0,4.540312e-08,-21.5,0.0,4.540312e-08,-6.5,0.0,-0.000407,-1.0,0.0,0.000204,0.5,0.0,0.000204,-0.5,0.0,98806.050,921721.500,941661.0,13347.000,921721.500,0.193358,0,0,0,1,0,0,0,0,0,0,0,0
310743,1,-4.082483e-01,0.0,0.0,-4.082483e-01,0.0,0.0,-4.082483e-01,0.0,0.0,-0.408248,0.0,0.0,-0.408248,0.0,0.0,2.041241,-5.0,0.0,7002.720,67864.230,75024.0,4.230,67864.230,0.000061,0,1,1,1,0,0,0,0,2,0,1,0
394447,2,5.295619e-01,-0.5,0.0,6.115381e-01,-7.0,0.0,-3.183548e-01,-7.0,0.0,-0.229629,-6.5,0.0,-0.180184,-8.5,0.0,-0.412932,0.0,0.0,17141.445,129132.000,129132.0,0.000,129132.000,0.000000,1,0,0,2,0,1,1,0,0,1,0,0


In [45]:
pa_ip.to_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_ip_all.csv")

## PA_CC

In [46]:
# pa_cc

In [47]:
n_heads.remove("NAME_TYPE_SUITE")

In [48]:
df = pa_cc
df = df.drop(n_heads + o_heads, axis=1)
df.head()

,SK_ID_CURR,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,DAYS_DECISION,NAME_TYPE_SUITE,CNT_PAYMENT,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,CC_BALANCE_30,CC_PAYMENT_30,CC_DPD_30,CC_BALANCE_60,CC_PAYMENT_60,CC_DPD_60,CC_BALANCE_90,CC_PAYMENT_90,CC_DPD_90,CC_BALANCE_120,CC_PAYMENT_120,CC_DPD_120,CC_BALANCE_150,CC_PAYMENT_150,CC_DPD_150,CC_BALANCE_180,CC_PAYMENT_180,CC_DPD_180
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000018,394447,2250.0,45000.0,45000.0,NaN,45000.0,NaN,NaN,NaN,-176,Unaccompanied,0.0,-175.0,-130.0,365243.0,365243.0,365243.0,-1.206367,-1.389108,0.0,1.434002,1.008387,0.0,0.992963,-0.429381,0.0,-0.349513,-0.292196,0.0,-0.415697,-0.215746,0.0,-0.455389,1.318045,0.0
1000030,361282,2250.0,45000.0,45000.0,NaN,45000.0,NaN,NaN,NaN,-255,Family,0.0,-243.0,-178.0,365243.0,365243.0,365243.0,0.946004,0.850318,0.0,0.828945,1.109609,0.0,0.861611,0.179508,0.0,-0.393110,-1.267179,0.0,-1.074370,0.280581,0.0,-1.169080,-1.152838,0.0
1000031,131335,2250.0,45000.0,45000.0,NaN,45000.0,NaN,NaN,NaN,-499,Family,0.0,-396.0,-362.0,365243.0,365243.0,365243.0,0.722068,-0.427910,0.0,0.719445,-0.427508,0.0,0.791575,-0.445538,0.0,0.204483,-0.215249,0.0,-0.817702,-0.514661,0.0,-1.619869,2.030866,0.0
1000035,436351,2250.0,45000.0,45000.0,NaN,45000.0,NaN,NaN,NaN,-178,Family,0.0,365243.0,365243.0,365243.0,365243.0,365243.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0
1000077,181153,2250.0,45000.0,45000.0,NaN,45000.0,NaN,NaN,NaN,-351,NaN,0.0,365243.0,365243.0,365243.0,365243.0,365243.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0


In [49]:
result = pd.DataFrame(index=pa_cc.SK_ID_CURR.unique())
result.index.name = "SK_ID_CURR"

### Gruppierung

In [50]:
# count
A = df[["SK_ID_CURR"]]
A.reset_index(level=0, inplace=True)
A = A.groupby(by=["SK_ID_CURR"]).count()
A.columns = ["CNT_CREDITS"]
result = pd.merge(result, A, how="left", left_index=True, right_index=True)

Anzahl der Kredite im CC-Datensatz je Kreditnehmer

In [51]:
cc.columns.values

array(['CC_BALANCE_30', 'CC_PAYMENT_30', 'CC_DPD_30', 'CC_BALANCE_60',
       'CC_PAYMENT_60', 'CC_DPD_60', 'CC_BALANCE_90', 'CC_PAYMENT_90',
       'CC_DPD_90', 'CC_BALANCE_120', 'CC_PAYMENT_120', 'CC_DPD_120',
       'CC_BALANCE_150', 'CC_PAYMENT_150', 'CC_DPD_150', 'CC_BALANCE_180',
       'CC_PAYMENT_180', 'CC_DPD_180'], dtype=object)

In [52]:
# means
mean_heads = list(cc.columns.values)

for head in mean_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).mean()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

In [53]:
# sums
sum_heads = ["AMT_ANNUITY", "AMT_APPLICATION", "AMT_CREDIT", "AMT_GOODS_PRICE"]

for head in sum_heads:
    A = df[["SK_ID_CURR", head]]
    A = A.fillna(0)
    A = A.groupby(by=["SK_ID_CURR"]).sum()
    result = pd.merge(result, A, how="left", left_index=True, right_index=True)

In [54]:
result.head()

,CNT_CREDITS,CC_BALANCE_30,CC_PAYMENT_30,CC_DPD_30,CC_BALANCE_60,CC_PAYMENT_60,CC_DPD_60,CC_BALANCE_90,CC_PAYMENT_90,CC_DPD_90,CC_BALANCE_120,CC_PAYMENT_120,CC_DPD_120,CC_BALANCE_150,CC_PAYMENT_150,CC_DPD_150,CC_BALANCE_180,CC_PAYMENT_180,CC_DPD_180,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_GOODS_PRICE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,
394447,1,-1.206367,-1.389108,0.0,1.434002,1.008387,0.0,0.992963,-0.429381,0.0,-0.349513,-0.292196,0.0,-0.415697,-0.215746,0.0,-0.455389,1.318045,0.0,2250.0,45000.0,45000.0,45000.0
361282,1,0.946004,0.850318,0.0,0.828945,1.109609,0.0,0.861611,0.179508,0.0,-0.393110,-1.267179,0.0,-1.074370,0.280581,0.0,-1.169080,-1.152838,0.0,2250.0,45000.0,45000.0,45000.0
131335,1,0.722068,-0.427910,0.0,0.719445,-0.427508,0.0,0.791575,-0.445538,0.0,0.204483,-0.215249,0.0,-0.817702,-0.514661,0.0,-1.619869,2.030866,0.0,2250.0,45000.0,45000.0,45000.0
436351,1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2250.0,45000.0,45000.0,45000.0
181153,1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,2250.0,45000.0,45000.0,45000.0


### Zusammenführung und Speicherung der Daten

In [55]:
pa_cc = result

In [56]:
pa_cc = pd.merge(pa_cc, cats, how="left", left_index=True, right_index=True)

In [57]:
pa_cc = pa_cc.add_prefix("PA_")

In [58]:
pa_cc2 = pd.read_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_cc.csv",index_col=0)

In [59]:
pa_cc2 = pa_cc[pa_cc2.columns.values]

In [60]:
pa_cc.head()

,PA_CNT_CREDITS,PA_CC_BALANCE_30,PA_CC_PAYMENT_30,PA_CC_DPD_30,PA_CC_BALANCE_60,PA_CC_PAYMENT_60,PA_CC_DPD_60,PA_CC_BALANCE_90,PA_CC_PAYMENT_90,PA_CC_DPD_90,PA_CC_BALANCE_120,PA_CC_PAYMENT_120,PA_CC_DPD_120,PA_CC_BALANCE_150,PA_CC_PAYMENT_150,PA_CC_DPD_150,PA_CC_BALANCE_180,PA_CC_PAYMENT_180,PA_CC_DPD_180,PA_AMT_ANNUITY,PA_AMT_APPLICATION,PA_AMT_CREDIT,PA_AMT_GOODS_PRICE,PA_CNT_NAME_CONTRACT_TYPE_Cash loans,PA_CNT_NAME_CONTRACT_TYPE_Consumer loans,PA_CNT_NAME_CONTRACT_TYPE_Revolving loans,PA_CNT_NAME_CONTRACT_TYPE_XNA,PA_CNT_WEEKDAY_APPR_PROCESS_START_FRIDAY,PA_CNT_WEEKDAY_APPR_PROCESS_START_MONDAY,PA_CNT_WEEKDAY_APPR_PROCESS_START_SATURDAY,PA_CNT_WEEKDAY_APPR_PROCESS_START_SUNDAY,PA_CNT_WEEKDAY_APPR_PROCESS_START_THURSDAY,PA_CNT_WEEKDAY_APPR_PROCESS_START_TUESDAY,PA_CNT_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,PA_CNT_FLAG_LAST_APPL_PER_CONTRACT_N,PA_CNT_FLAG_LAST_APPL_PER_CONTRACT_Y,PA_CNT_NAME_CASH_LOAN_PURPOSE_Building a house or an annex,PA_CNT_NAME_CASH_LOAN_PURPOSE_Business development,PA_CNT_NAME_CASH_LOAN_PURPOSE_Buying a garage,PA_CNT_NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land,PA_CNT_NAME_CASH_LOAN_PURPOSE_Buying a home,PA_CNT_NAME_CASH_LOAN_PURPOSE_Buying a new car,PA_CNT_NAME_CASH_LOAN_PURPOSE_Buying a used car,PA_CNT_NAME_CASH_LOAN_PURPOSE_Car repairs,PA_CNT_NAME_CASH_LOAN_PURPOSE_Education,PA_CNT_NAME_CASH_LOAN_PURPOSE_Everyday expenses,PA_CNT_NAME_CASH_LOAN_PURPOSE_Furniture,PA_CNT_NAME_CASH_LOAN_PURPOSE_Gasification / water supply,PA_CNT_NAME_CASH_LOAN_PURPOSE_Hobby,PA_CNT_NAME_CASH_LOAN_PURPOSE_Journey,PA_CNT_NAME_CASH_LOAN_PURPOSE_Medicine,PA_CNT_NAME_CASH_LOAN_PURPOSE_Money for a third person,PA_CNT_NAME_CASH_LOAN_PURPOSE_Other,PA_CNT_NAME_CASH_LOAN_PURPOSE_Payments on other loans,PA_CNT_NAME_CASH_LOAN_PURPOSE_Purchase of electronic equipment,PA_CNT_NAME_CASH_LOAN_PURPOSE_Refusal to name the goal,PA_CNT_NAME_CASH_LOAN_PURPOSE_Repairs,PA_CNT_NAME_CASH_LOAN_PURPOSE_Urgent needs,PA_CNT_NAME_CASH_LOAN_PURPOSE_Wedding / gift / holiday,PA_CNT_NAME_CASH_LOAN_PURPOSE_XAP,PA_CNT_NAME_CASH_LOAN_PURPOSE_XNA,PA_CNT_NAME_CONTRACT_STATUS_Approved,PA_CNT_NAME_CONTRACT_STATUS_Canceled,PA_CNT_NAME_CONTRACT_STATUS_Refused,PA_CNT_NAME_CONTRACT_STATUS_Unused offer,PA_CNT_NAME_PAYMENT_TYPE_Cash through the bank,PA_CNT_NAME_PAYMENT_TYPE_Cashless from the account of the employer,PA_CNT_NAME_PAYMENT_TYPE_Non-cash from your account,PA_CNT_NAME_PAYMENT_TYPE_XNA,PA_CNT_CODE_REJECT_REASON_CLIENT,PA_CNT_CODE_REJECT_REASON_HC,PA_CNT_CODE_REJECT_REASON_LIMIT,PA_CNT_CODE_REJECT_REASON_SCO,PA_CNT_CODE_REJECT_REASON_SCOFR,PA_CNT_CODE_REJECT_REASON_SYSTEM,PA_CNT_CODE_REJECT_REASON_VERIF,PA_CNT_CODE_REJECT_REASON_XAP,PA_CNT_CODE_REJECT_REASON_XNA,PA_CNT_NAME_TYPE_SUITE_Children,PA_CNT_NAME_TYPE_SUITE_Family,PA_CNT_NAME_TYPE_SUITE_Group of people,PA_CNT_NAME_TYPE_SUITE_Other_A,PA_CNT_NAME_TYPE_SUITE_Other_B,"PA_CNT_NAME_TYPE_SUITE_Spouse, partner",PA_CNT_NAME_TYPE_SUITE_Unaccompanied,PA_CNT_NAME_CLIENT_TYPE_New,PA_CNT_NAME_CLIENT_TYPE_Refreshed,PA_CNT_NAME_CLIENT_TYPE_Repeater,PA_CNT_NAME_CLIENT_TYPE_XNA,PA_CNT_NAME_GOODS_CATEGORY_Additional Service,PA_CNT_NAME_GOODS_CATEGORY_Animals,PA_CNT_NAME_GOODS_CATEGORY_Audio/Video,PA_CNT_NAME_GOODS_CATEGORY_Auto Accessories,PA_CNT_NAME_GOODS_CATEGORY_Clothing and Accessories,PA_CNT_NAME_GOODS_CATEGORY_Computers,PA_CNT_NAME_GOODS_CATEGORY_Construction Materials,PA_CNT_NAME_GOODS_CATEGORY_Consumer Electronics,PA_CNT_NAME_GOODS_CATEGORY_Direct Sales,PA_CNT_NAME_GOODS_CATEGORY_Education,PA_CNT_NAME_GOODS_CATEGORY_Fitness,PA_CNT_NAME_GOODS_CATEGORY_Furniture,PA_CNT_NAME_GOODS_CATEGORY_Gardening,PA_CNT_NAME_GOODS_CATEGORY_Homewares,PA_CNT_NAME_GOODS_CATEGORY_House Construction,PA_CNT_NAME_GOODS_CATEGORY_Insurance,PA_CNT_NAME_GOODS_CATEGORY_Jewelry,PA_CNT_NAME_GOODS_CATEGORY_Medical Supplies,PA_CNT_NAME_GOODS_CATEGORY_Medicine,PA_CNT_NAME_GOODS_CATEGORY_Mobile,PA_CNT_NAME_GOODS_CATEGORY_Office Appliances,PA_CNT_NAME_GOODS_CATEGORY_Other,PA_CNT_NAME_GOODS_CATEGORY_Photo / Cinema Equipment,PA_CNT_NAME_GOODS_

In [61]:
pa_cc.to_csv(DATASET_DIR / "2. Datenaufbereitung" / "pa_cc_all.csv")